In [1]:
import os
import datetime
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile

def download_dataset(dataset_path):
    # Initialize Kaggle API
    api = KaggleApi()
    api.authenticate()
    
    # Define the download path and filename
    download_path = './'
    zip_file = f'{dataset_path.split("/")[-1]}.zip'

    # Download dataset files to the current working directory
    api.dataset_download_files(dataset_path, path=download_path, unzip=False)

    # Define the CSV filename with today's date extension
    today = datetime.datetime.now().strftime('%Y-%m-%d')
    csv_filename = f'data_{today}.csv'

    # Unzipping the file and rename the CSV with today's date
    with zipfile.ZipFile(os.path.join(download_path, zip_file), 'r') as z:
        # Extract all the contents into the directory
        z.extractall(download_path)
        # Assume the main file is named 'data.csv' and rename it
        os.rename(os.path.join(download_path, 'data.csv'),
                  os.path.join(download_path, csv_filename))

    print(f'Dataset downloaded and saved as {csv_filename}')


In [2]:
download_dataset('curiel/rtechnology-posts-and-comments')

Dataset URL: https://www.kaggle.com/datasets/curiel/rtechnology-posts-and-comments


FileExistsError: [WinError 183] Cannot create a file when that file already exists: './data.csv' -> './data_2024-04-24.csv'

In [ ]:
from google.cloud import storage
from google.oauth2 import service_account
# Initialize Google Cloud Storage client
def upload_to_gcs(bucket_name, blob_prefix, local_directory='./'):
    

    # Path to your service account key file
    service_account_path = 'gc_key.json'

    # Create a credentials object from the service account file
    credentials = service_account.Credentials.from_service_account_file(service_account_path)

    # Use these credentials to initialize a Google Cloud Storage client
    storage_client = storage.Client(credentials=credentials)
    #storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    # Get today's date to define the CSV file
    today = datetime.datetime.now().strftime('%Y-%m-%d')
    local_file = f'data_{today}.csv'
    local_file_path = os.path.join(local_directory, local_file)

    # Define the blob (file) path in the bucket
    blob_path = os.path.join(blob_prefix, local_file)
    blob = bucket.blob(blob_path)

    # Upload the file
    blob.upload_from_filename(local_file_path)
    print(f"Uploaded {local_file} to {blob_path}")

    # Delete the local file after successful upload
    #os.remove(local_file_path)
    #print(f"Deleted local file {local_file_path} after successful upload.")
    

# Use the function
bucket_name = 'technology_subreddit'
blob_prefix = 'tech_data'

# Call the function to upload the file
upload_to_gcs(bucket_name, blob_prefix)
